In [ ]:
import pandas as pd

csv_file = 'dataset/preprocessed/data_grade1.csv'
df1 = pd.read_csv(csv_file)
df1 = df1.drop(columns=['STUID'])
print(df1)

      ST0  ST14  ST15  ST1_1  ST1_2  ST2_3  ST2_4  ST2_5  ST2_6  ST3  ...  \
0       1  48.0     0      2      4      4      3      4      0    1  ...   
1       1  45.0     1      2      4      1      1      3      3    2  ...   
2       1  48.0     4      1      2      0      8      1      2    2  ...   
3       0  44.0     2      2      5      4      1      3      0    2  ...   
4       0  50.0     0      2      3      0      4      3      4    2  ...   
...   ...   ...   ...    ...    ...    ...    ...    ...    ...  ...  ...   
1927    1  55.0     4      2      3      1      8      3      4    1  ...   
1928    1  35.0     4      2      3      3      8      4      5    0  ...   
1929    1  62.0     0      2      2      1      3      3      1    2  ...   
1930    1  43.0     0      2      3      2      3      4      8    3  ...   
1931    1  50.0     4      3      3      3      0      8      3    0  ...   

      DB7_5  DB7_6  DB7_7  DB7_8  KOR_S  ENG_S  MATH_S  KOR_HIGH  ENG_HIGH 

In [ ]:
csv_file2 = 'dataset/preprocessed/data_grade2.csv'
df2 = pd.read_csv(csv_file2)
df2 = df2.drop(columns=['STUID'])
print(df2)

      ST_PREV_KOR_S  ST_PREV_ENG_S  ST_PREV_MATH_S  ST0   ST14  ST15  ST1_1  \
0              30.5           39.0            26.0    1  157.0  43.0      3   
1              51.5           61.0            26.0    1  165.0  58.0      4   
2              79.0           81.5            94.0    1  161.0  45.0      4   
3              90.0           77.0            60.0    1  158.0  52.0      2   
4              65.0           96.0           100.0    0  170.0  46.5      2   
...             ...            ...             ...  ...    ...   ...    ...   
2190           52.5           18.5            33.0    1  160.0  56.0      4   
2191           74.0           67.5            78.0    1  165.0  60.0      2   
2192           70.5           71.0            78.0    1  156.0  43.0      4   
2193           67.5           85.5           100.0    1  165.0  65.0      2   
2194           70.5           25.0            15.0    1  154.0  45.0      3   

      ST1_2  ST2_3  ST2_4  ...  DB7_5  DB7_6  DB7_7

In [ ]:
csv_file3 = 'dataset/preprocessed/data_grade3.csv'
df3 = pd.read_csv(csv_file3)
df3 = df3.drop(columns=['STUID'])
print(df3)

      ST_PREV_KOR_S  ST_PREV_ENG_S  ST_PREV_MATH_S  ST0   ST14  ST15  ST1_1  \
0              65.0           25.0            41.0    1  157.0  47.0      3   
1              44.0           50.0            33.0    1  153.0  70.0      4   
2              73.0           93.0           100.0    1  161.1  45.0      4   
3              92.0          100.0            96.0    1  157.0  47.0      2   
4              68.0           86.0            65.0    1  158.1  55.0      2   
...             ...            ...             ...  ...    ...   ...    ...   
2100           41.0           74.0            20.0    1  155.0  75.0      2   
2101           71.0           92.0            96.0    0  168.0  54.0      2   
2102           68.0          100.0            77.0    1  158.0  54.0      3   
2103           24.0           24.5            25.0    1  160.0  50.0      2   
2104            0.0           18.0             5.0    0  163.0  50.0      3   

      ST1_2  ST2_3  ST2_4  ...  DB7_5  DB7_6  DB7_7

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score, confusion_matrix

# 예측 함수 정의
def model_predict(model, df, subject):

    # 과목별 학업성취도 (0,1) 변수를 예측 변수 y로 지정
    y = df[subject]
    # 과목별 원점수 및 학업성취도(0,1) 변수 제거 후 나머지 변수 모두 예측을 위한 변수 x값으로 지정
    X = df.drop(columns=['KOR_HIGH', 'ENG_HIGH', 'MATH_HIGH', 'KOR_S', 'ENG_S', 'MATH_S'])

    # train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 학습
    model.fit(X_train, y_train)

    # 모델 예측
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # 평가지표
    accuracy = accuracy_score(y_test, y_pred)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    kappa = cohen_kappa_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)

    return accuracy, sensitivity, specificity, kappa, auc

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(random_state=42)
grade_list = [df1, df2, df3]
subject_list = ['KOR_HIGH', 'ENG_HIGH', 'MATH_HIGH']
subject_name = ['국어 과목', '영어 과목', '수학 과목']

# 랜덤포레스트 수행
print("RANDOM FOREST RESULT")

grade_count = 0
subject_count = 0

# 1,2,3 학년별로 출력
for grade in grade_list:
    print('학년 : ', grade_count+1)
    subject_count = 0
    # 과목별로
    for subject in subject_list:
        # 5가지 점수 계산, model_predict()에 첫번째 변수에 모델명 수정시 해당 모델로 계산 진행됨
        accuracy, sensitivity, specificity, kappa, auc = model_predict(random_forest_model, grade, subject)
        print(f"Subject Name: {subject_name[subject_count]}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Sensitvity: {sensitivity:.4f}")
        print(f"Specificity: {specificity:.4f}")
        print(f"Kappa: {kappa:.4f}")
        print(f"AUC: {auc:.4f} \n")
        subject_count = subject_count + 1
    grade_count = grade_count+1

RANDOM FOREST RESULT
학년 :  1
Subject Name: 국어 과목
Accuracy: 0.6563
Sensitvity: 0.7418
Specificity: 0.5517
Kappa: 0.2971
AUC: 0.7068 

Subject Name: 영어 과목
Accuracy: 0.7829
Sensitvity: 0.7887
Specificity: 0.7772
Kappa: 0.5659
AUC: 0.8563 

Subject Name: 수학 과목
Accuracy: 0.8165
Sensitvity: 0.8597
Specificity: 0.7590
Kappa: 0.6230
AUC: 0.8836 

학년 :  2
Subject Name: 국어 과목
Accuracy: 0.7062
Sensitvity: 0.7860
Specificity: 0.6071
Kappa: 0.3981
AUC: 0.7638 

Subject Name: 영어 과목
Accuracy: 0.8337
Sensitvity: 0.8093
Specificity: 0.8621
Kappa: 0.6675
AUC: 0.8974 

Subject Name: 수학 과목
Accuracy: 0.8109
Sensitvity: 0.7798
Specificity: 0.8416
Kappa: 0.6217
AUC: 0.8749 

학년 :  3
Subject Name: 국어 과목
Accuracy: 0.7173
Sensitvity: 0.7736
Specificity: 0.6603
Kappa: 0.4342
AUC: 0.8036 

Subject Name: 영어 과목
Accuracy: 0.8195
Sensitvity: 0.7731
Specificity: 0.8683
Kappa: 0.6397
AUC: 0.8966 

Subject Name: 수학 과목
Accuracy: 0.8100
Sensitvity: 0.6935
Specificity: 0.9144
Kappa: 0.6146
AUC: 0.8753 

